In [31]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from skimage.feature import local_binary_pattern
from skimage.color import rgb2lab, rgb2hsv
import pywt
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
from scipy.stats import spearmanr, pearsonr
from scipy import ndimage
from skimage.feature import hog, graycomatrix, graycoprops
from sklearn.tree import DecisionTreeRegressor

import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)


In [32]:
train_df = pd.read_csv("../../dataset/ratings/train.csv")
test_df = pd.read_csv("../../dataset/ratings/test.csv")
val_df = train_df.iloc[-90:].copy()
val_df.shape


(90, 2)

In [33]:
train_df = train_df.iloc[:-90].copy()
train_df.shape


(420, 2)

In [34]:
train_df.head()


,filename,MOS
0,f329.png,0.134646
1,f316.png,-0.954147
2,f257.png,0.978417
3,f206.png,-3.041686
4,f459.png,-0.192770


In [35]:
val_df.head()


,filename,MOS
420,f68.png,-0.149500
421,f191.png,-0.241829
422,f278.png,0.154798
423,f62.png,0.469404
424,f164.png,0.634564


In [36]:
test_df.head()


,filename,MOS
0,f244.png,0.681995
1,f126.png,0.709697
2,r3.png,1.223393
3,f145.png,-1.211034
4,f170.png,-0.696548


In [37]:
X_train = list(train_df["filename"])
y_train = list(train_df["MOS"])

X_valid = list(val_df["filename"])
y_valid = list(val_df["MOS"])

X_test = list(test_df["filename"])
y_test = list(test_df["MOS"])


In [38]:
from pathlib import Path
import numpy as np
from PIL import Image
# … all your other imports stay the same …

class HandcraftedRealnessPredictor:
    def __init__(
        self,
        train_root: str,
        test_root: str,
        max_depth: int = 5,
        random_state: int = SEED,
    ):
        """
        Parameters
        ----------
        train_root : str
            Folder that contains *both* training and validation images.
        test_root : str
            Folder that contains test images.
        """
        self.train_root = Path(train_root)
        self.test_root  = Path(test_root)
        self.model = DecisionTreeRegressor(max_depth=max_depth,
                                           random_state=random_state)

    # ------------------------------------------------------------------ #
    # --------------------  feature extraction ------------------------- #
    # ------------------------------------------------------------------ #
    def extract_features(self, image_name: str, phase: str = "train") -> np.ndarray:
        """
        Extract 45 handcrafted features from an RGB image (no resizing).

        Parameters
        ----------
        image_name : str
            File name (relative to the chosen root) or a full path.
        phase : {"train", "test"}
            Selects which root directory to look in.
        """
        if phase not in {"train", "test"}:
            raise ValueError("phase must be 'train' or 'test'")

        root = self.train_root if phase == "train" else self.test_root
        img_path = root / image_name   # pathlib handles the join cleanly
        image = np.asarray(Image.open(img_path))

        gray = np.mean(image, axis=2)
        h, w = gray.shape
        features = []

        # ---------- 1. Enhanced colour features (8) ----------
        lab = rgb2lab(image)
        hsv = rgb2hsv(image)
        lab_a = lab[:, :, 1].astype(np.float32) / 127
        lab_b = lab[:, :, 2].astype(np.float32) / 127
        features.extend([
            np.std(lab_a),
            np.std(lab_b),
            hsv[:, :, 0].std(),
            np.percentile(hsv[:, :, 1], 95),
            np.mean(np.abs(lab_a)),
            np.mean(np.abs(lab_b)),
            np.median(hsv[:, :, 2]),
            (hsv[:, :, 2] > 0.9).mean(),
        ])

        # ---------- 2. Advanced edge features (10) ----------
        sobel_x = ndimage.sobel(gray, axis=1)
        sobel_y = ndimage.sobel(gray, axis=0)
        edge_magnitude = np.hypot(sobel_x, sobel_y)
        features.extend([
            edge_magnitude.mean(),
            edge_magnitude.std(),
            np.percentile(edge_magnitude, 90), # Strong edge presence
            (edge_magnitude > 0.2).mean(), # Edge density
            # Edge regularity
            np.corrcoef(sobel_x[::10].flatten(), sobel_y[::10].flatten())[0, 1],
            # Edge direction consistency
            np.std(np.arctan2(sobel_y, sobel_x)[edge_magnitude > 0.1]),
            # Asymmetry (AI artifacts often symmetric)
            np.abs(sobel_x - np.fliplr(sobel_x)).mean(),
            np.abs(sobel_y - np.flipud(sobel_y)).mean(),
            # Edge sharpness
            np.mean(edge_magnitude[edge_magnitude > 0.05]),
            np.max(edge_magnitude),
        ])

        # ---------- 3. Texture & pattern (15) ----------
        glcm = graycomatrix(
            (gray * 255).astype(np.uint8),
            distances=[1, 3],
            angles=[0, np.pi / 4],
            levels=256,
            symmetric=True,
        )
        for prop in ["contrast", "dissimilarity", "homogeneity", "energy", "correlation"]:
            features.extend(graycoprops(glcm, prop).flatten())
        
        hog_feats = hog(gray,
                        orientations=8,
                        pixels_per_cell=(32, 32),
                        cells_per_block=(1, 1),
                        visualize=False)
        features.extend([hog_feats.mean(), hog_feats.std()])

        # ---------- 4. Frequency & compression (12) ----------
        coeffs = pywt.wavedec2(gray, "db2", level=3)
        for cH, cV, cD in coeffs[1:]:
            features.extend([
                np.abs(cH).mean(),
                np.abs(cV).mean(),
                np.abs(cD).mean(),
                np.percentile(np.abs(cH), 90),
            ])

        # ---------- 5. Perceptual grouping (two extras) ----------
        contrast_energy = ndimage.gaussian_filter(gray, sigma=1) - \
                          ndimage.gaussian_filter(gray, sigma=3)
        features.append(np.mean(contrast_energy ** 2))

        saliency = np.abs(ndimage.gaussian_gradient_magnitude(gray, sigma=2))
        features.extend([saliency.mean(), saliency.std()])

        return np.array(features)

    # ------------------------------------------------------------------ #
    # ------------------------  model API  ----------------------------- #
    # ------------------------------------------------------------------ #
    def fit(self, X, y, phase: str = "train"):
        feats = np.array([self.extract_features(img, phase=phase) for img in X])
        self.model.fit(feats, y)
        return self

    def predict(self, X, phase: str = "train"):
        feats = np.array([self.extract_features(img, phase=phase) for img in X])
        return self.model.predict(feats)


### Initialize and train


In [39]:
predictor = HandcraftedRealnessPredictor(
    train_root="../../dataset/images/train_images/",   
    test_root="../../dataset/images/test_images/"          
)

predictor.fit(X_train, y_train, phase="train")


#### predict on val


In [40]:
# ----- Validation (still inside train_root) -----
val_preds = predictor.predict(X_valid, phase="train")
score_r, _ = spearmanr(val_preds, y_valid)
score_p, _ = pearsonr(val_preds, y_valid)

print(score_r, score_p)


0.5504170661969076 0.5073133014002407


#### predict on test


In [41]:
# ----- Testing (different root) -----
test_preds = predictor.predict(X_test, phase="test")

score_r, _ = spearmanr(test_preds, y_test)
score_p, _ = pearsonr(test_preds, y_test)


print(score_r, score_p)


0.49781117350971243 0.43373095406300893
